In [ ]:
from pycocotools.coco import COCO
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt

# Change this according to your data path
DATA_ROOT = '/home/naro/images_hackaton/'
data_dir = os.path.join(DATA_ROOT, 'bildacher/images')
ann_file= os.path.join(DATA_ROOT, 'bildacher/coco/output.json')

seg_output_path = os.path.join(DATA_ROOT, 'bildacher/masks')
coco = COCO(ann_file)
catIds = coco.getCatIds(catNms=['rumex_leaf'])
imgIds = coco.getImgIds(catIds=catIds )
if not os.path.exists(seg_output_path):
    os.mkdir(seg_output_path)


# A loop that run over all images and extract the masks

In [ ]:
for i in range(len(imgIds)):
    img = coco.loadImgs(imgIds[i])[0]
    annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=0)
    anns = coco.loadAnns(annIds)
    mask = coco.annToMask(anns[0])
    for i in range(len(anns)):
        mask = coco.annToMask(anns[i])
        file_name = os.path.join(data_dir,img['file_name'])
        try:
            im = cv2.imread(file_name)
            original_img = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
            mask = mask.astype(np.uint8)
            result = cv2.bitwise_and(original_img, original_img, mask=mask)
        except:
            print('Image {} unable to be loaded'.format(img['file_name']))

        try:
            first_non_zero_row = np.where(np.any(mask != 0, axis=1))[0][0]
            last_non_zero_row = np.where(np.any(mask != 0, axis=1))[0][-1]
            first_non_zero_col = np.where(np.any(mask != 0, axis=0))[0][0]
            last_non_zero_col = np.where(np.any(mask != 0, axis=0))[0][-1]
            cropped_image = original_img[first_non_zero_row:last_non_zero_row + 1, first_non_zero_col:last_non_zero_col + 1]
            path_name = img['file_name'].split('.')[0] + '_' + str(i) + '.png'
            cv2.imwrite(os.path.join(seg_output_path,path_name),cropped_image)
            print("processing...")

        except:
            print('Non sucessful extraction {}'.format(img['file_name']))


# A go through Example on one image

In [ ]:
i = 5
img = coco.loadImgs(imgIds[i])[0]
annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=0)
anns = coco.loadAnns(annIds)
mask = coco.annToMask(anns[0])
file_name = os.path.join(data_dir,img['file_name'])
im = cv2.imread(file_name)
original_img = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
plt.imshow(original_img)

In [ ]:
plt.imshow(mask)

In [ ]:
mask = mask.astype(np.uint8)
result = cv2.bitwise_and(original_img, original_img, mask=mask)
plt.imshow(result)

In [ ]:
first_non_zero_row = np.where(np.any(mask != 0, axis=1))[0][0]
last_non_zero_row = np.where(np.any(mask != 0, axis=1))[0][-1]
first_non_zero_col = np.where(np.any(mask != 0, axis=0))[0][0]
last_non_zero_col = np.where(np.any(mask != 0, axis=0))[0][-1]

In [ ]:
cropped_image = original_img[first_non_zero_row:last_non_zero_row + 1, first_non_zero_col:last_non_zero_col + 1]

In [ ]:
plt.imshow(cropped_image)